In [1]:
import secrets
import random
import math

Tools to find modular inverse

In [2]:
class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class ModuloError(Error):
    """Exception raised for errors in the input.

    Attributes:
        message -- explanation of the error
    """

    def __init__(self, message):
        self.message = message

In [3]:
def egcd(a, b):
    '''ax + by = g = gcd(a, b)
       returns (g, x, y)'''
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

In [4]:
def modinv(a, n):
    '''Finding inverse of a mod n'''
    g, x, y = egcd(a, n)
    if abs(g) != 1:
        raise ModuloError("Inverse doesn't exist")
    else:
        return x % n

In [6]:
modinv(65537, )

46703207172867055440606820351900522440782435780355529386658723515136428255651018779627790318472064623024002458727091824305101189693338174273839942594927884289

Prime numbers from 2 to 1000 to pre-check of generated numbers before miller-rabin test

In [5]:
possible_div = [3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97
                   ,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,179
                   ,181,191,193,197,199,211,223,227,229,233,239,241,251,257,263,269
                   ,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,367
                   ,373,379,383,389,397,401,409,419,421,431,433,439,443,449,457,461
                   ,463,467,479,487,491,499,503,509,521,523,541,547,557,563,569,571
                   ,577,587,593,599,601,607,613,617,619,631,641,643,647,653,659,661
                   ,673,677,683,691,701,709,719,727,733,739,743,751,757,761,769,773
                   ,787,797,809,811,821,823,827,829,839,853,857,859,863,877,881,883
                   ,887,907,911,919,929,937,941,947,953,967,971,977,983,991,997]

Geberates random numbers of length bit_len with first_bit ==1. That guarantees that the number will have bit_len bits.

In [6]:
def generate_random_num(bit_len):
    random_num = secrets.randbits(bit_len) #generate random int of length bit_len bits
    random_num |= 1 << (bit_len - 1)
    return random_num

Primarity test

In [7]:
def miller_rabin(p, k=5):
    #step 0
    #finding p-1 = d * 2^s 
    d = p-1
    s = 0
    while(d % 2 == 0):
        d//=2
        s+=1
    def check_witness(x):
        x_pow_d = pow(x, d, p)
        if (x_pow_d == 1) or (x_pow_d == p - 1):
            return True
        #2.2 for r in [1,s-1]  check:
        xr_cache = x_pow_d
        for r in range(1, s):
            xr = pow(xr_cache,2, p)
            if xr == (p-1):
                return True
            elif xr == 1:
                #print("P is not strong pseudoprime to base {}".format(x))
                return False
            xr_cache = xr
            #print("P is not strong pseudoprime to base {}".format(x))
        return False
    
    for i in range(k):
        #step 1 
        #choosing x (natural) from 'interval (2,p) independently from previous x
        x = random.randrange(2, p) 
        # finding gcd(x, p) using euclidian algo
        g = math.gcd(x, p)
        # if gcd = 1, go to step 2. Else: p isn't prime, quit
        if (g > 1):
            return False
        #step 2
        #checking if p is strong pseudoprime to base x
        #2.1 if x^d mod p == +-1, p is strong pseudoprime to base x, else go to step 2.2
        if (check_witness(x) == 0):
            return False        
    return True

In [124]:
p = generate_random_num(10)
print(p)
miller_rabin(p)

981


False

Checks if given number is prime

In [15]:
def is_prime(random_num):
    for div in possible_div:
            if (random_num % div) == 0:
                break
    if (miller_rabin(random_num, 5)):
            return True
    else:
            return False

Generates prime numbers of bit_len

In [16]:
def generate_prime(bit_len):
    while(True):
        random_num = generate_random_num(bit_len)
        for div in possible_div:
            if (random_num % div) == 0:
                break
        if (miller_rabin(random_num, 5)):
            return random_num

In [17]:
generate_prime(1024)

110727536460453745377273474661969016805954703172504266356961188273692796039049977424324636478303055673006323824123549956297652920156829989931460416398663123790796828599637520070774612458086848104696386742374462855856207343988703740647461262308232642067599156199240510922169899681726928425382768282472785422123

Generates number p where p-1 has large prime divisors

In [32]:
def good_rsa_prime(bit_len):
    prime = generate_prime(bit_len)
    for i in range(1,1000):
        if is_prime(2*i*prime + 1):
            prime = 2*i*prime + 1
            return prime
    return good_rsa_prime(bit_len)

In [33]:
good_rsa_prime(1024)

325122933329646928110477705175332141816687786051176947460769685757805150610419477111513067251925444275849060222140951509216375113936155282278184879931059322599553089216942058573967408180704985506574790349583815783568082829144389133309133164068630138746455294236613489312558940146366033903774589867869121515489287

Generating primes p i q of length bit_len. There are two possible variants:
    - We have public key(n1,e1) of receiver B. So we have to generate public(n,e)
    and private key (d,p,q) for sender A. The rule for keys: n1>=n. If n1<n, regenerate p and q for sender A until n1>=n. Call function create_A_key(bit_len, n1)
    -We generate keys for both A and B. The rule for keys: n1>=n. If n1<n, swap places keys of A and B. Create function create_A_B_keys(bit_len).

Generating 4 prime numbers and making sure p*q <= p1*q1

In [35]:
def create_A_B_keys(bit_len):
    p, p1, q1, q = good_rsa_prime(bit_len), good_rsa_prime(bit_len), good_rsa_prime(bit_len), good_rsa_prime(bit_len)
    if p*q > p1*q1:
        p, p1, q, q1 = p1, p, q1, q
    assert p*q <= p1*q1
    return p, q, p1, q1

In [36]:
create_A_B_keys(512)

(2921369837006012005082503126520214648547366891362904454226476118082167097602724783847462958220745451700733125576881193941653245871483421033145756319660310373,
 1699872748748085977964004751008721922801017786726792608431720970031003669291202199097915426157313343255965346626100883853317037770102239983885048731810306487,
 6150025848628223747747529921960864842951059894270269710888296212059229885830176333738186947379976327037952282262975639841405415149234187599035799210723335269,
 12183822664090472700056454995819212278759941006457324730039557821912142980761436062639239347290035177766607074592876568676455461074577837355516538038034368547)

In [40]:
n_test = 2921369837006012005082503126520214648547366891362904454226476118082167097602724783847462958220745451700733125576881193941653245871483421033145756319660310373 * 1699872748748085977964004751008721922801017786726792608431720970031003669291202199097915426157313343255965346626100883853317037770102239983885048731810306487 

In [38]:
def create_A_key(bit_len, n1):
    n = math.inf
    while(n1<n):
        p, q = good_rsa_prime(bit_len), good_rsa_prime(bit_len)
        n = p*q
    return p, q

In [41]:
create_A_key(512, n_test )

(1660606862936201377532266556395437386909937582668619353964781647603799352005108935526029657260594561725751998083238607623182119165612394351047628716853424409,
 1452328117707243421873431048013614411049862739274875268142959049924161102134488333446757572470233843081565423571032115422882688776326777285193610662798783401)

GenerateKeyPair procedure for RSA, creates public and private keys for one abonent

In [42]:
def GenerateKeyPair(p, q):
    n = p * q
    phi = (p-1) * (q-1)
    e = pow(2, 16) + 1
    if (math.gcd(e, q-1) != 1 or math.gcd(e,p-1)!=1):
        e = generate_prime(17)
    assert e>2 and e <= (phi - 1)
    assert math.gcd(e, phi) ==1
    d = modinv(e, phi)
    private_key = d, p, q
    public_key = n, e
    return public_key, private_key

In [44]:
p, q, p1, q1 = create_A_B_keys(512)

Generate public and private keys for abonent A

In [49]:
public_key_A, private_key_A = GenerateKeyPair(p, q)

Generate public and private keys for abonent B

In [50]:
public_key_B, private_key_B = GenerateKeyPair(p, q)

Creates message of random hex values, if hexi=True turns decimal value to hex string

In [60]:
def GenerateMessage(bit_len, hexi=False):
    M = generate_random_num(bit_len)
    return hex(M) if hexi else M

In [61]:
 GenerateMessage(300, True)

'0x9316eca5a751722d3f4b91103074c2c4e80e1afd69a9ffd94621d8781e417ce017506a0da94'

In [62]:
def Encrypt(M, public_key):
    n, e = public_key
    C = pow(M, e, n)
    return C

In [63]:
def Decrypt(C, private_key, n):
    d, p, q = private_key
    M = pow(C, d, n)
    return M

In [64]:
def Sign(M, private_key, n):
    d, p, q = private_key
    S = pow(M, d, n)
    return S

In [65]:
def Verify(M, S, public_key):
    n, e = public_key
    return M == pow(S, e, n)


Testing procedures

In [67]:
M =  GenerateMessage(300, False)

In [68]:
C_A = Encrypt(M, public_key_A)
C_A

401293440586674101090015747396028130876339686567517740009709497022645425548081817974070830172020391841633385721814710027986193271287794223469906039040813742167180132202755206705753277589661939307305106898620737915559196383376205171884432350804667978126530098013978180678587438617398157768782066560346009031231266

In [69]:
D_A = Decrypt(C_A, private_key_A, public_key_A[0])
D_A

1901635554202948125109111944544541164476206632461322671629713304778335602540702489362581017

In [70]:
M == D_A

True

User A sends key to user B

In [71]:
def SendKey(k, public_key_A, public_key_B, private_key_A):
    #if key A or key B are hexidecimal, turn them into decimal
    if (isinstance(public_key_A[0], str) and isinstance(private_key_A[0], str)) :
        public_key_A = int(public_key_A[0], 16), int(public_key_A[1], 16)
        private_key_A = int(private_key_A[0], 16), int(private_key_A[1], 16)
    if (isinstance(public_key_B[0], str)):
        public_key_B = int(public_key_B[0], 16), int(public_key_B[1], 16) 
    if (isinstance(k, str)):
        k = int(k, 16)
    n = public_key_A[0]
    n1 = public_key_B[0]
    if (n1 < n):
        raise KeyError('n1 < n')
    k1 = Encrypt(k, public_key_B)
    S = Sign(k, private_key_A, n)
    S1 = Encrypt(S,public_key_B)
    return k1, S1

In [75]:
k = generate_random_num(256)
print(k)

58329245861941685948984821242862652901014889433787903090589214073258759175113


In [76]:
k1, S1 =  SendKey(k, public_key_A, public_key_B, private_key_A)
k1, S1

(275794615897152184901468985957229147002397602149028377878591790025139360613241061447273622379240471181301326002735311126701983206167891760666350489774517186924348928881860370746196592087294818085879097342004108498355991096750706772657856892173744610695941224222624282964756149572354156796811386818205928680206260,
 58329245861941685948984821242862652901014889433787903090589214073258759175113)

In [77]:
hex(k1), hex(S1)

('0x5fe2891d79691cb4d127a5d031fdd1e1e85b6efcd3d077aea144c3617ee9b9a9de56c3d92e18062e2e902e00f954b35892255111ef9ede81cf8e6240f1c0e70d9176b4efca5b7989dfcdfb4cdf11f892733c7f3cda1b12621055a87e2688a4fb167d20206c3db4a4c97a88ead0724d30ca61c64ff5a6e3150bb646d6d4229c22fb4',
 '0x80f52ee729258d171508c93d30bc49dcf8763c7ca8c62f91e75c352764943bc9')

User B receives key from A

In [78]:
def ReceiveKey(k1, S1, public_key_B, private_key_B, public_key_A):
    n1 = public_key_B[0]
    k = Decrypt(k1, private_key_B, n1)
    S = Decrypt(S1, private_key_B, n1)
    assert Verify(k, S, public_key_A)
    print('verified')
    return k

In [81]:
k = ReceiveKey(k1, S1, public_key_B, private_key_B, public_key_A)
k

verified


58329245861941685948984821242862652901014889433787903090589214073258759175113